
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# Lab: Rollout Strategies with Workflows

In this lab, you will do the following:
1. Bring in a table from Databricks Marketplace.
1. Perform feature engineering and store the table in Databricks Feature Store.
1. Perform validation test to check for missing values, confirm the schema, and check for non-negative values. 
1. You will also check that normalization was carried out correctly. 
1. Finally, you will train and serve 2 models while directing traffic to one model 40% of the time and to the other model 60% of the time. 

**Learning Objectives**

By the end of this lab you will be able to
- Demonstrate knowledge of how to setup integration tests with Databricks Workflows.
- Demonstrate knowledge of how to configure a model serving endpoint for serving an uneven traffic distribution for 2 models. 
- Demonstrate an understanding of working with MLflow and Unity Catalog for testing rollout strategies.

## REQUIRED - SELECT CLASSIC COMPUTE
Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.
Follow these steps to select the classic compute cluster:
1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.
1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:
   - In the drop-down, select **More**.
   - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:
1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.
1. Find the triangle icon to the right of your compute cluster name and click it.
1. Wait a few minutes for the cluster to start.
1. Once the cluster is running, complete the steps above to select your cluster.


## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **17.3.x-cpu-ml-scala2.13**


## Classroom Setup

To get into the lesson, you first need to build some data assets and define some configuration variables required for this demonstration. When running the following cell, the output is hidden so our space isn't cluttered. To view the details of the output, you can hover over the next cell and click the eye icon. 

The cell after the setup, titled `View Setup Variables`, displays the various variables that were created. You can click the Catalog icon in the notebook space to the right to see that your catalog was created with no data.

In [0]:
%run ../Includes/Classroom-Setup-2.Lab

The following variables will be useful for you to configure your various parameters.

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")

# Workflow Setup
Here you will setup an integration test using Databrick Workflows. There will be a total of 5 Workflow notebooks that you will need to **fill out** as a part of completing this lab. They are located in the folder title **2.4 Lab - Model Training Pipeline**.

### Review of How to Attach Workflow Notebooks

> **For Experienced Users:** If you are already familiar with setting up workflows, you can skip the detailed steps below and proceed directly to setting up the workflow.
> 
> **For New Users or Those Needing a Refresher:** Follow the step-by-step instructions below to set up your workflow.


1. Navigate to **Jobs & Pipelines**:
   - Open the [**Jobs & Pipelines**](/jobs) page (also accessible from the left sidebar).
1. Click on **Create** and select **Job** from the dropdown in the upper-right corner of the page.

1. Set the Job name to ***Rollout Strategies with Workflow*** or something similar for easy identification.

1. **Set Up the Task**:
   - Select `Notebook`.
   - Select **Notebook** at the top of the menu that appears.
   - Provide a **Task Name** (e.g., `Silver_to_Feature_Store`).
   - Set the **Type** to `Notebook`.
   - For **Source**, select `Workspace`.
   - In **Path**, navigate to the second notebook.

1. **Configure Compute**:
   - In the **Compute** section, select the cluster you have been working on.

1. **Configure Dependencies**:
   - In the **Depends On** section, notice that your previous task is automatically selected. Leave this as is.

1. **Add Parameters**

1. Click **Save task**

1. in the **Tasks** grid, click **+ Add task**. Repeat the steps till third task for the third notebook.

1. **Create and Run the Task**:
   - Once all details and parameters are entered, click on **Create Task**.
   - After creating the task, click **Run Now** to execute it.

Wait a moment for the pipeline to run and validate that the job was successful.

###Step 1: Set Up Workflow Notebooks  

The first step is to setup notebooks that will be used for Workflows. All Workflows notebooks can be found in the folder labeled **2.4 Lab - Model Training Pipeline**.

For each notebook below, navigate to **`2.4 Lab - Model Training Pipeline`**, open the specified notebook, and fill in the necessary code. **Widgets have been preconfigured**, so you only need to focus on implementing the required logic.  


1. Configure Notebook: **"01 Silver to Feature Store"** 
- Enter the task name of your choice, or alternatively, use the name of the notebook. 
- Navigate to **`2.4 Lab - Model Training Pipeline`** and open **`01 - Silver to Feature Store`**.  
- Fill in the required code where necessary.  
- Use the following parameters when setting up the Workflow:  

```
{
  "catalog": "dbacademy",
  "column": "Age",
  "primary_key": "id",
  "schema": "<your_schema>",
  "silver_table_name": "diabetes",
  "target_column": "Diabetes_binary"
}
```


2. Configure Notebook: **"02 Data Validation Tests"**  
- Enter the task name of your choice, or alternatively, use the name of the notebook.
- Navigate to **`2.4 Lab - Model Training Pipeline`** and open **`02 - Data Validation Tests`**.  
- Fill in the required code where necessary.  
- Use the following parameters when setting up the Workflow:  

```
{
  "catalog": "dbacademy",
  "schema": "<your_schema>",
  "silver_table_name": "diabetes"
}
```

3. Configure Notebook: **"03 Normalization Validation"**  
- Enter the task name of your choice, or alternatively, use the name of the notebook.
- Navigate to **`2.4 Lab - Model Training Pipeline`** and open **`03 - Normalization Validation`**.  
- Fill in the required code where necessary.  
- Use the following parameters when setting up the Workflow:  

```
{
  "catalog": "dbacademy",
  "schema": "<your_schema>",
  "normalized_column": "Age",
  "silver_table_name": "diabetes"
}
```

4. Configure Notebook: **"04 Train Models on Validated Features"**  
- Enter the task name of your choice, or alternatively, use the name of the notebook.
- Navigate to **`2.4 Lab - Model Training Pipeline`** and open **`04 - Train Models on Validated Features`**.  
- Fill in the required code where necessary.  
- Use the following parameters when setting up the Workflow:  

```
{
  "catalog": "dbacademy",
  "delete_column": "BMI",
  "primary_key": "id",
  "schema": "<your_schema>",
  "silver_table_name": "diabetes",
  "target_column": "Diabetes_binary"
}
```


5. Configure Notebook: **"05 Serve the Model"**  
- Enter the task name of your choice, or alternatively, use the name of the notebook.
- Navigate to **`2.4 Lab - Model Training Pipeline`** and open **`05 - Serve the Model`**.  
- Fill in the required code where necessary.  
- Use the following parameters when setting up the Workflow:  

```
{
  "catalog": "dbacademy",
  "schema": "<your_schema>"
}
```

### Step 2: Individual Notebook Testing

After you have configured each of the notebooks, you should run them individually to squash any outstanding bugs. Waiting for an entire workflow to kick off just to have the final notebook fail can be a huge waste of time.



### Step 3: Chain the notebooks together to complete the model serving workflow. 

See the notes before Step 1 if you need assistance doing this.

## Conclusion

In this lab, you created a workflow that demonstrates common testing strategies with a goal of training 2 models and serving them with Mosaic AI Model Serving. You learned how integral test strategies are the role MLflow and Unity Catalog play in accomplishing tasks involving testing. Finally, you demonstrated knowledge of splitting traffic in a 40/60 -split by configuring deployments with Mosaic AI Model Serving.

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>